In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ddot import Ontology
import networkx as nx
from sklearn.preprocessing import StandardScaler

In [19]:
def load_hierarchy_network():
    
    network_name = '../data/NeST/NeST'
    
    nodes_df = pd.read_csv(network_name + '_node.csv')[['name', 'Annotation', 'Genes']]
    
    edges_df = pd.read_csv(network_name + '_edge.sif', sep='\t', header=None, names = ['S', 'M', 'T'])
    
    return nodes_df, edges_df

In [3]:
def convert_to_clixo_format(hierarchy_nodes_df, hierarchy_edges_df, gene_list, n_type):
    
    ont = pd.DataFrame()
    temp_file = '../data/' + n_type + '.txt'
    ont_file = '../data/NeST_' + n_type + '_ont.txt'
    
    for _, row in hierarchy_edges_df.iterrows():
        ont = ont.append({'Source' : row['S'], 'Target' : row['T'], 'Mapping' : 'default'}, ignore_index=True)

    for _, row in hierarchy_nodes_df.iterrows():
        if pd.isna(row['Genes']):
            continue
        genes = row['Genes'].split()
        for gene in genes:
            if gene in gene_list:
                ont = ont.append({'Source' : row['name'], 'Target' : gene, 'Mapping' : 'gene'}, ignore_index=True)

    ont = ont[['Source', 'Target', 'Mapping']]
    
    ont.to_csv(temp_file, sep='\t', header=False, index=False)
    ont = Ontology.from_table(temp_file, clixo_format=True)
    os.remove(temp_file)
    
    ont.propagate(direction='reverse', inplace=True)
    
    ont = ont.collapse_ontology(method='python')
    
    ont.to_table(ont_file, clixo_format=True)
    
    return ont

In [4]:
def create_gene_mutation_feature(gene_list, cell_mutation_matrix):
    g_m = {}        
    cell_mutation_matrix = cell_mutation_matrix.transpose()
    for i, gene in enumerate(gene_list):
        g_m[gene] = np.sum(cell_mutation_matrix[i])/float(len(cell_mutation_matrix[i]))
        
    return g_m

In [5]:
def get_subsystem_count(gene, hierarchy_nodes_df):
    counter = 0
    for _,row in hierarchy_nodes_df.iterrows():
        if pd.isna(row['Genes']):
            continue
        if gene in row['Genes']:
            counter += 1
    
    return counter

In [6]:
def create_gene_subsystem_feature(hierarchy_nodes_df, hierarchy_edges_df, gene_list):
    dG = nx.DiGraph()
    for _,row in hierarchy_edges_df.iterrows():
        dG.add_edge(row['S'], row['T'])
    leaf_nodes = [n for n in dG.nodes() if dG.out_degree(n) == 0]
    
    g_s = {}
    for gene in gene_list:
        g_s[gene] = get_subsystem_count(gene, hierarchy_nodes_df)/float(len(hierarchy_nodes_df))
        
    return g_s, leaf_nodes

In [7]:
def create_feature_matrix(gene_list, g_m, g_s):
    feature_df = pd.DataFrame()
    for g in gene_list:
        feature_df = feature_df.append({'G' : g, 'M' : g_m[g], 'S' : g_s[g]}, ignore_index=True)
    
    return feature_df

In [8]:
def score_subsystems(hierarchy_nodes_df, gene_list, feature_df):
    s_score = {}
    for _,row in hierarchy_nodes_df.iterrows():
        if pd.isna(row['Genes']):
            continue
        genes = row['Genes'].split()
        num = 0
        den = 0
        for g in genes:
            if g not in gene_list:
                continue
            gm = float(feature_df[feature_df.G == g]['M'])
            gs = float(feature_df[feature_df.G == g]['S'])
            num += gs * math.sqrt(gm)
            den += gs
        if num == 0:
            s_score[row['name']] = 0
        else:
            s_score[row['name']] = num/den
    
    return s_score

In [20]:
n_type = 'clinical_trial'

gene_list = list(pd.read_csv('../data/gene2ind_' + n_type + '.txt', sep='\t', header=None, names=['I', 'G'])['G'])
cell_mutation = np.loadtxt('../data/cell2mutation_' + n_type + '.txt', delimiter=',')

h_nodes_df, h_edges_df = load_hierarchy_network()

In [21]:
g_m = create_gene_mutation_feature(gene_list, cell_mutation)
g_s, leaf_subsystems = create_gene_subsystem_feature(h_nodes_df, h_edges_df, gene_list)

In [22]:
feature_df = create_feature_matrix(gene_list, g_m, g_s)

In [23]:
s_score = score_subsystems(h_nodes_df, gene_list, feature_df)

In [24]:
leaf_subsystem_score = {}
for ls in leaf_subsystems:
    leaf_subsystem_score[ls] = s_score[ls]
leaf_subsystem_score = {s:sc for s,sc in sorted(leaf_subsystem_score.items(), key=lambda item:item[1])}

In [25]:
low_score_systems = [sys for sys in leaf_subsystem_score.keys() if leaf_subsystem_score[sys] < 0.001]
low_score_df = h_nodes_df[h_nodes_df.name.isin(low_score_systems)]
for _,row in low_score_df.iterrows():
    gene_count = len(row['Genes'].split())
    if gene_count > 10:
        low_score_systems.remove(row['name'])

In [26]:
len(low_score_systems)

82

In [27]:
h_nodes_df = h_nodes_df.query("name not in @low_score_systems")
h_edges_df = h_edges_df.query("T not in @low_score_systems")

In [28]:
ont = convert_to_clixo_format(h_nodes_df, h_edges_df, gene_list, n_type)

In [29]:
print(ont)

326 genes, 194 terms, 835 gene-term relations, 234 term-term relations
node_attributes: []
edge_attributes: []


In [49]:
#Randomize the genes

ont2_file = '../data/NeST_' + n_type + '_bb_ont.txt'
ont2 = ont.shuffle_genes()
ont2.propagate(direction='reverse', inplace=True)
ont2 = ont2.collapse_ontology(method='python')
ont2.to_table(ont2_file, clixo_format=True)
print(ont2)